# Estimating Coordinated Daily Activity Patterns

This notebook illustrates how to re-estimate ActivitySim's CDAP model. This process 
includes running ActivitySim in estimation mode to read household travel survey files and write out
the estimation data bundles used in this notebook.  To review how to do so, please visit the other
notebooks in this directory.

# Load libraries

In [1]:
import numpy as np
import pandas as pd
import larch
import os

In [4]:
os.chdir('test')

# Load data and prep model for estimation

In [5]:
modelname = "cdap"

from activitysim.estimation.larch import component_model
model, data = component_model(modelname, return_data=True)

one-hot encoding choice array
converting data_ch to <class 'numpy.float64'>
one-hot encoding choice array
converting data_ch to <class 'numpy.float64'>
one-hot encoding choice array
converting data_ch to <class 'numpy.float64'>
one-hot encoding choice array
converting data_ch to <class 'numpy.float64'>
one-hot encoding choice array
converting data_ch to <class 'numpy.float64'>


# Review Inputs

## Settings

In [6]:
data.settings

┣          INDIV_AND_HHSIZE1_SPEC: 'cdap_indiv_and_hhsize1.csv'
┣                    COEFFICIENTS: 'cdap_coefficients.csv'
┣        INTERACTION_COEFFICIENTS: 'cdap_interaction_coefficients.csv'
┣ FIXED_RELATIVE_PROPORTIONS_SPEC: 'cdap_fixed_relative_proportions.csv'
┣                       CONSTANTS: ┣       FULL: 1
┃                                  ┣       PART: 2
┃                                  ┣ UNIVERSITY: 3
┃                                  ┣    NONWORK: 4
┃                                  ┣    RETIRED: 5
┃                                  ┣    DRIVING: 6
┃                                  ┣     SCHOOL: 7
┃                                  ┣  PRESCHOOL: 8
┣                 PERSON_TYPE_MAP: ┣ WORKER: [1, 2]
┃                                  ┣  CHILD: [6, 7, 8]
┣                annotate_persons: ┣ SPEC: 'annotate_persons_cdap'
┃                                  ┣   DF: 'persons'
┣             annotate_households: ┣   SPEC: 'annotate_households_cdap'
┃                         

## Households

In [7]:
data.households

,household_id,home_zone_id,income,hhsize,HHT,auto_ownership,num_workers,sample_rate,income_in_thousands,income_segment,...,hh_work_auto_savings_ratio,num_under16_not_at_school,num_travel_active,num_travel_active_adults,num_travel_active_preschoolers,num_travel_active_children,num_travel_active_non_preschoolers,participates_in_jtf_model,joint_tour_frequency,num_hh_joint_tours
0,166,71,9000,1,6,0,0,1,9.00,1,...,0.0,0,1,1,0,0,1,False,0_tours,0
1,197,80,4400,1,6,0,0,1,4.40,1,...,0.0,0,1,1,0,0,1,False,0_tours,0
2,268,91,1200,1,4,1,0,1,1.20,1,...,0.0,0,1,1,0,0,1,False,0_tours,0
3,375,105,0,1,6,1,0,1,0.00,1,...,0.0,0,1,1,0,0,1,False,0_tours,0
4,387,112,15770,1,6,1,0,1,15.77,1,...,0.0,0,1,1,0,0,1,False,0_tours,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,2863464,5,0,1,0,1,0,1,0.00,1,...,0.0,0,1,1,0,0,1,False,0_tours,0
1996,2863483,20,0,1,0,1,0,1,0.00,1,...,0.0,0,1,1,0,0,1,False,0_tours,0
1997,2863806,85,0,1,0,1,0,1,0.00,1,...,0.0,0,1,1,0,0,1,False,0_tours,0
1998,2864518,115,0,1,0,1,0,1,0.00,1,...,0.0,0,1,1,0,0,1,False,0_tours,0


## Person Data

In [8]:
data.person_data

,person_id,model_choice,override_choice,household_id,age,PNUM,sex,pemploy,pstudent,ptype,...,COLLFTE,COLLPTE,TOPOLOGY,TERMINAL,household_density,employment_density,density_index,is_cbd,chunk_id,cdap_rank
0,166,N,N,166,54,1,2,3,3,4,...,0.00000,0.00000,1,3.21263,24.783133,31.566265,13.883217,False,0,1
1,197,N,N,197,46,1,2,3,3,4,...,0.00000,0.00000,1,3.68156,56.783784,10.459459,8.832526,False,1,1
2,268,N,N,268,46,1,1,3,3,4,...,3598.08521,0.00000,1,3.29100,11.947644,45.167539,9.448375,True,2,1
3,375,N,N,375,54,1,2,3,3,4,...,0.00000,0.00000,1,4.11499,73.040169,28.028350,20.255520,True,3,1
4,387,N,N,387,44,1,2,3,3,4,...,227.78223,41.22827,1,3.83527,26.631579,45.868421,16.848945,False,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4400,7554799,N,N,2863464,93,1,2,3,3,5,...,72.14684,0.00000,1,5.52555,38.187500,978.875000,36.753679,False,1995,1
4401,7554818,N,N,2863483,68,1,1,3,3,5,...,0.00000,0.00000,3,3.99027,39.838272,71.693001,25.608291,True,1996,1
4402,7555141,N,N,2863806,93,1,2,3,3,5,...,0.00000,0.00000,1,4.27539,51.675676,47.216216,24.672699,False,1997,1
4403,7555853,N,N,2864518,71,1,1,3,3,5,...,0.00000,0.00000,1,25.52083,15.938148,551.353820,15.490363,True,1998,1


## interaction_coef

In [9]:
data.interaction_coef

,activity,interaction_ptypes,coefficient,cardinality,slug
0,H,11,1.6260,2,HH11
1,H,12,0.7407,2,HH12
2,H,13,1.1830,2,HH13
3,H,14,0.9436,2,HH14
4,H,15,1.2980,2,HH15
...,...,...,...,...,...
128,N,,-1.3460,4,NNNN
129,H,,-3.7330,4,HHHH
130,M,,-1.5280,5,MMMMM
131,N,,-3.4530,5,NNNNN


## Spec 1

In [10]:
data.spec1.head()

,Description,Expression,M,N,H
0,Full-time worker alternative-specific constants,ptype == 1,coef_full_time_worker_asc_M,coef_full_time_worker_asc_N,NaN
1,Part-time worker alternative-specific constants,ptype == 2,coef_part_time_worker_asc_M,coef_part_time_worker_asc_N,NaN
2,University student alternative-specific constants,ptype == 3,coef_university_student_asc_M,coef_university_student_asc_N,NaN
3,Non-working adult alternative-specific constants,ptype == 4,coef_UNAVAILABLE,coef_non_working_adult_asc_N,NaN
4,Retired alternative-specific constants,ptype == 5,coef_UNAVAILABLE,coef_retired_asc_N,NaN


# Review Model

## Utility Functions for HHSize 1

In [11]:
model[0].utility_functions() ## HH Size = 1 

<xmle.Elem 'div' with 1 children>

## Utility Functions for HHSize 2

In [12]:
model[1].utility_functions() ## HH Size = 2

<xmle.Elem 'div' with 1 children>

# Re-Estimate Model

With the model setup for estimation, the next step is to estimate the model coefficients.  Make sure to use a sufficiently large enough household sample and set of zones to avoid an over-specified model, which does not have a numerically stable likelihood maximizing solution.  Larch has two built-in estimation methods: BHHH and SLSQP.  BHHH is the default and typically runs faster, but does not follow constraints on parameters.  SLSQP is safer, but slower, and may need additional iterations.

In [13]:
r = model.estimate(method='SLSQP', options={'maxiter':1000})

,value,initvalue,nullvalue,minimum,maximum,holdfast,note,best
coef_UNAVAILABLE,-999.000000,-999.000000,0.0,NaN,NaN,True,,-999.000000
coef_child_who_is_in_school_or_too_young_for_school_interaction_with_off_peak_accessibility_to_retail_N,0.856930,0.082330,0.0,NaN,NaN,False,,0.856930
coef_driving_age_child_who_is_in_school_asc_M,0.738480,2.330919,0.0,NaN,NaN,False,,0.738480
coef_driving_age_child_who_is_in_school_asc_N,-8.930886,-0.599119,0.0,NaN,NaN,False,,-8.930886
coef_driving_age_child_who_is_in_school_interaction_income_between_50k_and_100k_H,-0.938531,-0.503100,0.0,NaN,NaN,False,,-0.938531
...,...,...,...,...,...,...,...,...
coef_N_124_122_144,0.996710,0.349100,0.0,NaN,NaN,False,,0.996710
coef_N_166,-1.976101,0.355300,0.0,NaN,NaN,False,,-1.976101
coef_N_222_224_444,-2.046233,-1.386000,0.0,NaN,NaN,False,,-2.046233
coef_N_246_226_446,-0.859376,-0.857100,0.0,NaN,NaN,False,,-0.859376


<ipython-input-13-4499ea85d59e>:1: PossibleOverspecification: WARNING: Model is possibly over-specified (hessian is nearly singular).
  r = model.estimate(method='SLSQP', options={'maxiter':1000})
/Users/jeffnewman/OneDrive - Cambridge Systematics/Git/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 3.7787895067513534e-43 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
<ipython-input-13-4499ea85d59e>:1: RuntimeWarning: invalid value encountered in sqrt
  r = model.estimate(method='SLSQP', options={'maxiter':1000})


In [14]:
model.parameter_summary()

,Value,Std Err,t Stat,Signif,Like Ratio,Null Value
coef_UNAVAILABLE,-999.,0.00,NA,,NA,0.00
coef_child_who_is_in_school_or_too_young_for_school_interaction_with_off_peak_accessibility_to_retail_N,0.857,0.618,1.39,,NA,0.00
coef_driving_age_child_who_is_in_school_asc_M,0.738,0.649,1.14,,NA,0.00
coef_driving_age_child_who_is_in_school_asc_N,-8.93,6.17,-1.45,,NA,0.00
coef_driving_age_child_who_is_in_school_interaction_income_between_50k_and_100k_H,-0.939,0.760,-1.23,,NA,0.00
coef_driving_age_child_who_is_in_school_interaction_with_fewer_cars_than_workers_H,0.409,0.669,0.61,,NA,0.00
coef_driving_age_child_who_is_in_school_interaction_with_income_more_than_100k_H,-1.75,0.887,-1.97,*,NA,0.00
coef_driving_age_child_who_is_in_school_interaction_with_less_than_20k_H,0.456,0.807,0.57,,NA,0.00
coef_full_time_worker_asc_M,1.03,2.64,0.39,,NA,0.00
coef_full_time_worker_asc_N,0.742,0.188,3.95,***,NA,0.00


# Output Estimation Results

In [15]:
from larch.util.activitysim import update_coefficients
coefficients = update_coefficients(model, data)

In [16]:
result_dir = data.edb_directory/'estimated'

In [17]:
os.makedirs(result_dir, exist_ok=True)

### Write the re-estimated coefficients file

In [18]:
coefficients.reset_index().to_csv(
    result_dir/"cdap_coefficients_revised.csv", 
    index=False,
)

### Write the model estimation report, including coefficient t-statistic and log likelihood

In [19]:
for hh_size, submodel in enumerate(model, 1):
    if hh_size > 3: 
        break 
        # the largest HH sizes have massive model reports that 
        # Excel doesn't handle well; review them in Jupyter 
        # instead if you are interested
    submodel.to_xlsx(
        result_dir/f"cdap_model_estimation_hhsize{hh_size}.xlsx", 
        data_statistics=False,
    )

# Next Steps

The final step is to either manually or automatically copy the `cdap_coefficients.csv_revised.csv` file to the configs folder, rename it to `cdap_coefficients.csv.csv`, and run ActivitySim in simulation mode.

In [20]:
pd.read_csv(result_dir/"cdap_coefficients_revised.csv")

,coefficient_name,value,constrain
0,coef_UNAVAILABLE,-999.000000,T
1,coef_full_time_worker_asc_M,1.034987,F
2,coef_full_time_worker_asc_N,0.742053,F
3,coef_part_time_worker_asc_M,4.900043,F
4,coef_part_time_worker_asc_N,0.659214,F
...,...,...,...
161,coef_N_xxxx,-1.346000,F
162,coef_H_xxxx,-3.733000,F
163,coef_M_xxxxx,-0.040214,F
164,coef_N_xxxxx,-0.825691,F
